In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 whi

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
from datasets import load_dataset

In [4]:
dataset=load_dataset('hafidikhsan/c4_200m-gec-train100k-test25k')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/555 [00:00<?, ?B/s]

(…)-00000-of-00001-44bfcc14e6298a2d.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

(…)-00000-of-00001-d91ef46575827981.parquet:   0%|          | 0.00/5.12M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'length_input', 'length_output'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['input', 'output', 'length_input', 'length_output'],
        num_rows: 25000
    })
})

In [6]:
dataset=dataset.remove_columns(['length_input','length_output'])

In [7]:
dataset1=dataset

In [8]:
dataset1

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 25000
    })
})

In [9]:
from transformers import T5TokenizerFast, TFT5ForConditionalGeneration

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [10]:
tokenizer=T5TokenizerFast.from_pretrained('t5-small')

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [11]:
def preprocess_function(examples):
  inputs=[i for i in examples['input']]
  targets=[i for i in examples['output']]
  model_inputs=tokenizer(inputs,text_target=targets,max_length=128,truncation=True)
  return model_inputs

In [12]:
tokenized_dataset=dataset1.map(preprocess_function,batched=True)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [13]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['input', 'output', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 25000
    })
})

In [14]:
def my_preprocess_function(data):
  input=tokenizer(data['input'],max_length=128,truncation=True)
  target=tokenizer(data['output'],max_length=128,truncation=True)
  return {
      'input_ids':input['input_ids'],
      'attention_mask':input['attention_mask'],
      'labels':target['input_ids']
  }


In [15]:
  tokenized_2nd_dataset=dataset.map(my_preprocess_function,batched=True)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [16]:
tokenized_2nd_dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['input', 'output', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 25000
    })
})

In [17]:
tokenized_dataset['train'][0]

{'input': 'Bitcoin is for $7,094 this morning, which CoinDesk says.',
 'output': 'Bitcoin goes for $7,094 this morning, according to CoinDesk.',
 'input_ids': [9310,
  19,
  21,
  11301,
  6,
  4198,
  591,
  48,
  1379,
  6,
  84,
  15589,
  2962,
  7,
  157,
  845,
  5,
  1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [9310,
  1550,
  21,
  11301,
  6,
  4198,
  591,
  48,
  1379,
  6,
  1315,
  12,
  15589,
  2962,
  7,
  157,
  5,
  1]}

In [18]:
tokenized_2nd_dataset['train'][0]

{'input': 'Bitcoin is for $7,094 this morning, which CoinDesk says.',
 'output': 'Bitcoin goes for $7,094 this morning, according to CoinDesk.',
 'input_ids': [9310,
  19,
  21,
  11301,
  6,
  4198,
  591,
  48,
  1379,
  6,
  84,
  15589,
  2962,
  7,
  157,
  845,
  5,
  1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [9310,
  1550,
  21,
  11301,
  6,
  4198,
  591,
  48,
  1379,
  6,
  1315,
  12,
  15589,
  2962,
  7,
  157,
  5,
  1]}

In [19]:
tokenized_dataset=tokenized_dataset.remove_columns(['input','output'])
tokenized_2nd_dataset=tokenized_2nd_dataset.remove_columns(['input','output'])

In [20]:
model=TFT5ForConditionalGeneration.from_pretrained('t5-small')
from transformers import DataCollatorForSeq2Seq
data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer,model=model,return_tensors='tf')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [21]:
tf_train_set=tokenized_2nd_dataset['train'].to_tf_dataset(
    columns=['input_ids','attention_mask','labels'],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,

)

tf_test_set=tokenized_2nd_dataset['test'].to_tf_dataset(
    columns=['input_ids','attention_mask','labels'],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,

)

In [22]:
for i in tf_train_set.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(16, 68), dtype=int64, numpy=
array([[ 3538,  3891,  3914, ...,     0,     0,     0],
       [  493, 19252,     5, ...,     0,     0,     0],
       [    3, 26586,   138, ...,     0,     0,     0],
       ...,
       [ 1521,    25,   479, ...,     0,     0,     0],
       [  363,   656,    39, ...,     0,     0,     0],
       [  389,     7,   138, ...,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(16, 68), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>, 'labels': <tf.Tensor: shape=(16, 63), dtype=int64, numpy=
array([[ 3538,  3891,  3914, ...,  -100,  -100,  -100],
       [  493, 19252,    22, ...,  -100,  -100,  -100],
       [    3, 26586,   138, ...,  -100,  -100,  -100],
       ...,
       [ 1521,    25,   479, ...,  -100,  -100,  -100],
       [  363,   656

In [23]:
for i in tf_train_set.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(16, 121), dtype=int64, numpy=
array([[10476,   228,  5260, ...,     0,     0,     0],
       [   27,   183,     3, ...,     0,     0,     0],
       [19099,     7,    54, ...,     0,     0,     0],
       ...,
       [  276, 22777,    39, ...,     0,     0,     0],
       [ 1203,     6,    36, ...,     0,     0,     0],
       [ 1318, 21367,  7772, ...,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(16, 121), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>, 'labels': <tf.Tensor: shape=(16, 116), dtype=int64, numpy=
array([[10476,    54,  5260, ...,  -100,  -100,  -100],
       [   27,   183,     3, ...,  -100,  -100,  -100],
       [19099,     7,    54, ...,  -100,  -100,  -100],
       ...,
       [  276, 22777,    39, ...,  -100,  -100,  -100],
       [ 1203,   

In [24]:
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  16449536  
                                                                 
 encoder (TFT5MainLayer)     multiple                  35330816  
                                                                 
 decoder (TFT5MainLayer)     multiple                  41625344  
                                                                 
Total params: 60506624 (230.81 MB)
Trainable params: 60506624 (230.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    metrics=['accuracy']
)

In [26]:
model.fit(
    tf_train_set,
    validation_data=tf_test_set,
    epochs=2
)

Epoch 1/2
6250/6250 [==============================] - 1857s 291ms/step - loss: 0.8655 - accuracy: 0.3230 - val_loss: 0.7292 - val_accuracy: 0.3334
Epoch 2/2
6250/6250 [==============================] - 1818s 291ms/step - loss: 0.7836 - accuracy: 0.3261 - val_loss: 0.7059 - val_accuracy: 0.3348


In [28]:
input="Haves you done I homework?"
tokenized_input=tokenizer([input],return_tensors='tf')
out=model.generate(**tokenized_input,max_length=128)
print(out)

tf.Tensor([[    0  2114    25   612    82 11920    58     1]], shape=(1, 8), dtype=int32)


In [29]:
print(tokenizer.decode(out[0],skip_special_tokens=True))

Have you done my homework?
